## law_1.docx
- pinecone store 저장
    - index name: 임의
- RetrievalQA 구현
    - prompt: rlm/rag-prompt
    - 질문: 전세사기

In [7]:
import os

from dotenv import load_dotenv
from langchain import hub
from langchain_community.document_loaders import Docx2txtLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pinecone import Pinecone


## 환경변수 읽어오기
load_dotenv()
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
LANGCHAIN_API_KEY = os.getenv('LANGCHAIN_API_KEY')

## 문서 읽고 분할
# loader = Docx2txtLoader('law_1.docx')

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=1500,
#     chunk_overlap=200,
# )

# document_list = loader.load_and_split(text_splitter=text_splitter)

## 임베딩 -> 백터 스토어(데이터베이스)에 저장
## 임베딩 모델 지정
embedding = OpenAIEmbeddings(model="text-embedding-3-large")
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = 'law-quize01-index'

## 파이콘: 저장
# database = PineconeVectorStore.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     index_name='law-quize01-index',
# )

## 저장된 인덱스 가져오기
## [방법 1]
# database = PineconeVectorStore(
#     index=pc.Index(index_name),
#     embedding=embedding,
# )

## [방법 2]
database = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding,
)

## RetrievalQA
llm = ChatOpenAI(model='gpt-4o')
prompt = hub.pull('rlm/rag-prompt')

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_chain =  (
    {
        'context': database.as_retriever() | format_docs,
        'question':RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

# qa_chain.invoke('전세사기피해자 금융지원에 대해 설명해주세요.')
qa_chain.invoke('전세사기피해자 임대인을 알려주세요')

'전세사기피해자 임대인은 전세사기피해자법에 정의된 "임대인등"에 포함됩니다. "임대인등"은 임대인, 임대인의 대리인, 임대인의 공인중개사, 임차인 모집자, 다수 임대인의 배후에 있는 동일인, 그리고 이에 경제적 이익을 공유하는 조직 등을 포함합니다. 이들이 전세사기 피해의 임대인으로 여겨질 수 있습니다.'

In [8]:
import os

from dotenv import load_dotenv
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone


## 환경변수 읽어오기 ########################################################
load_dotenv()
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
LANGCHAIN_API_KEY = os.getenv('LANGCHAIN_API_KEY')

## 백터 스토어(데이터베이스)에서 인덱스 가져오기 #############################
## 임베딩 모델 지정
embedding = OpenAIEmbeddings(model="text-embedding-3-large")
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = 'law-quize01-index'

## 저장된 인덱스 가져오기 
database = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding,
)

## RetrievalQA #################################################################
llm = ChatOpenAI(model='gpt-4o')
prompt = hub.pull('rlm/rag-prompt')

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_chain =  (
    {
        'context': database.as_retriever() | format_docs,
        'question':RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

qa_chain.invoke('전세사기피해자 금융지원에 대해 설명해주세요.')
# qa_chain.invoke('전세사기피해자 임대인을 알려주세요')

'전세사기피해자에 대한 금융지원은 국가와 지방자치단체가 전세사기 피해자에게 필요한 자금을 융자하거나 지원하는 것을 포함합니다. 주택도시기금에서 전세사기 피해자에게 주택의 임대차에 필요한 자금을 융자할 수 있습니다. 또한, 금융회사는 피해자의 전세 관련 대출에 대한 채무 불이행 등록을 유예하거나 삭제할 수 있는 지원을 제공합니다.'